In [87]:
from pymodbus.constants import Endian
from collections import OrderedDict
from pymodbus.payload import BinaryPayloadDecoder, BinaryPayloadBuilder
from pymodbus.client import ModbusTcpClient

client = ModbusTcpClient('127.0.0.1', 5020)
builder = BinaryPayloadBuilder(byteorder=Endian.Big, wordorder=Endian.Little)
builder.add_16bit_float(12.34)
builder.add_16bit_float(-12.34)
registers = builder.to_registers()
print("Writing Registers:")
print(registers)
print("\n")
payload = builder.build()
address = 151
count = 4
print(count)
rr = client.read_holding_registers(address, count, slave=1)
assert not rr.isError()
print(rr.registers)

decoder = BinaryPayloadDecoder.fromRegisters(rr.registers, Endian.Big, Endian.Little)
decoded = OrderedDict([("16float", round(decoder.decode_32bit_float(),2)), ("16float2", round(decoder.decode_32bit_float(),2))])
print("Decoded Data")
for name, value in iter(decoded.items()):
    print(f"{name}\t{hex(value) if isinstance(value, int) else value}")
print("\n")
client.close()

Writing Registers:
[18988, 51756]


4
[50856, 16809, 0, 0]
Decoded Data
16float	21.22
16float2	0.0




In [ ]:
def build_payload():


In [20]:
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')
future = producer.send('quickstart', key=b'test_key', value=b'test')
result = future.get(30)
producer.flush()

ImportError: cannot import name 'IncompatibleBrokerVersion' from 'kafka.errors' (C:\Users\hungm\PycharmProjects\Modbus2Kafka\venv\lib\site-packages\kafka\errors.py)

In [3]:
from confluent_kafka import Producer
import socket

conf = {'bootstrap.servers': '127.0.0.1:9092', 'client.id': socket.gethostname()}
producer = Producer(conf)
def asked(err, msg):
    if err is not None:
        print('Failed to deliver message %s: %s' % (str(msg), str(err)))
    else:
        print('Message produced: %s' % (str(msg)))
producer.produce('quickstart',value='Mike', callback=asked)
producer.flush(12)

1

In [17]:
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadDecoder
from pymodbus.client import ModbusTcpClient
import pandas as pd
from kafka import KafkaProducer
import json


def get_param():
    data =  pd.read_excel(r"CauHinhThongSoPaChien.xlsx")
    return data
configModbus = json.load(open("CauHinhModbus.json"))
client = ModbusTcpClient(configModbus['host'], configModbus['port'])

params = get_param()
values = []
for index, row in params.iterrows():
    address = params.at[index, 'Addr']
    count = params.at[index, 'Rid']
    rr = client.read_holding_registers(address, count, slave=configModbus['slaveId'])
    assert not rr.isError()
    decoder = BinaryPayloadDecoder.fromRegisters(rr.registers, Endian.Big, Endian.Little)
    if params.at[index, 'DataType'] == 'bit_float_32':
        values.append(round(decoder.decode_32bit_float(), 4))
    elif params.at[index, 'DataType'] == 'bit_undecimal_16':
        values.append(decoder.decode_8bit_uint())
params['Values'] = values

kafkaConfig = json.load(open("CauHinhKafka.json"))
producer = KafkaProducer(bootstrap_servers=kafkaConfig['bootstrapServers'],api_version=(0, 10, 0),value_serializer=lambda v: json.dumps(v).encode('utf-8'))

producer.send(kafkaConfig['topic'], params[['Name', 'Description', 'DataType', 'Values']].to_json())
f = open("value.json", "w")
f.write(params[['Name', 'Description', 'DataType', 'Values']].to_json())
f.close()
print('gen data')

gen data


In [7]:
from kafka import KafkaProducer
import json
producer = KafkaProducer(bootstrap_servers=['localhost:29092'],api_version=(0, 10, 0),value_serializer=lambda v: json.dumps(v).encode('utf-8'))
for _ in range(100):
    producer.send('foobar', 'some_message_bytes')

In [2]:
from schedule import every, repeat, run_pending
import time

@repeat(every(5).seconds)
def job():
    print("I am a scheduled job")

while True:
    run_pending()
    time.sleep(1)

I am a scheduled job
I am a scheduled job
I'm working...
I am a scheduled job
I am a scheduled job
I am a scheduled job
I am a scheduled job
I am a scheduled job


KeyboardInterrupt: 